## 1. Install local (and custom) version of transformers

In [ ]:
% cd transformers-master/
! python3 setup.py install

## 2. Create TSV

In [ ]:
import pandas as pd


def convertTSV(f, placeholder):
    t = dict()
    for line in f.readlines():
        l = line.split()
        if placeholder:
            if len(l) == 2:
                if l[0] == "meta" and "lang" not in l[1]:
                    n = l[1]
                    s = "neutral"
                    t[n] = {"label":s, "text":[]}
                else:
                    t[n]["text"].append(l[0])

        else:
            if len(l) == 3:
                n = l[1]
                s = l[2]
                t[n] = {"label":s, "text":[]}
            if len(l) == 2:
                t[n]["text"].append(l[0])
    return t


def fileWrite(t, f):
    label_dict = {"negative":0, "neutral":1, "positive":2}
    for i in range(len(t)):
        j = str(i+1)
        if i == 0 and f.name.split("/")[-1] == "dev.tsv":
            f.write("{}\t{}\n".format(" ".join(t[j]["text"]), label_dict[t[j]["label"]]))
        f.write("{}\t{}\n".format(" ".join(t[j]["text"]), label_dict[t[j]["label"]]))


with open("dev.conll", "r") as f:
    dev = convertTSV(f, False)
with open("train.conll", "r") as f:
    train = convertTSV(f, False)


with open("train.tsv", "w") as f:
    fileWrite(train, f)
with open("dev.tsv", "w") as f:
    fileWrite(dev, f)


print(len(dev))
print("Done!")

## 2. Fine-tune language model

In [ ]:
% transformers-master/examples/
# specify your own data directory here
! python3 run_glue.py --data_dir "." \
    --model_type xlmroberta \
    --model_name_or_path xlm-roberta-large \
    --task_name sent \
    --output_dir "./xlmr_out" \
    --overwrite_output_dir \
    --do_train \
    --adam_epsilon 1e-08 \
    --learning_rate 1e-05 \
    --num_train_epochs 3.0 \
    --save_steps 100000 \
    --do_eval

## 4. Evaluate

In [ ]:
% cd transformers-master/examples/
# specify your own data directory here
! python3 run_glue.py \
    --data_dir "." \
    --model_type xlm-roberta \
    --model_name_or_path "./xlmr_out" \
    --output_dir "./xlmr_out" \
    --overwrite_output_dir \
    --task_name sent \
    --do_eval